In [5]:
from pyscf import gto, scf
from scipy.linalg import fractional_matrix_power
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def controleer_orthonormaliteit(matrix):
    is_ortho = np.allclose(np.eye(len(matrix[0])), matrix)
    if is_ortho:
        return "De matrix is orthonormaal."
    else:
        return "De matrix is niet orthonormaal."

def roteer_en_verplaats_waterstof(theta1, afstand, theta2):
    # Verplaats en roteer waterstofatomen met de gegeven hoeken en afstand
    x1 = afstand * np.cos(np.radians(theta1))
    y1 = afstand * np.sin(np.radians(theta1))

    x2 = afstand * np.cos(np.radians(theta2))
    y2 = afstand * np.sin(np.radians(theta2))

    return f'O 0.0 0.0 0.0; H {x1} {y1} 0.0; H {x2} {y2} 0.0'

def rhf(atom):
    mol = gto.M(atom=atom, basis='ccpvdz')

    print(mol.nelec)
    MAXITER = 100000
    E_conv = 1.0e-6

    S = mol.intor('int1e_ovlp')
    T = mol.intor('int1e_kin')
    V = mol.intor('int1e_nuc')
    H_core = T + V
    eri = mol.intor('int2e')

    enuc = mol.get_enuc()
    ndocc = mol.nelec[0]

    # ==> Inspecting S for AO orthonormality <==
    controleer_orthonormaliteit(S)

    # ==> Construct AO orthogonalization matrix A <==
    A = fractional_matrix_power(S, -0.5)
    A = np.asarray(A)
    ASA = A @ S @ A

    # Check orthonormality
    controleer_orthonormaliteit(ASA)

    # ==> Compute C & D matrices with CORE guess <==
    # Transformed Fock matrix
    F_p = A @ H_core @ A

    # Diagonalize F_p for eigenvalues & eigenvectors with NumPy
    epsilon, C_p = np.linalg.eigh(F_p)

    # Transform C_p back into AO basis
    C = A @ C_p
    # Grab occupied orbitals
    C_occ = C[:, :ndocc]

    # Build density matrix from occupied orbitals
    D = np.einsum('ik, kj -> ij', C_occ, C_occ.T, optimize=True)

    # ==> SCF Iterations <==
    # Pre-iteration energy declarations
    SCF_E = 0.0
    E_old = 0.0

    # Begin Iterations
    for scf_iter in range(1, MAXITER + 1):
        # Build Fock matrix
        J = np.einsum('pqrs,rs->pq', eri, D, optimize=True)
        K = np.einsum('prqs,rs->pq', eri, D, optimize=True)

        F = H_core + 2 * J - K
        # Compute RHF energy
        SCF_E = enuc + np.sum(D * (H_core + F))

        # SCF Converged?
        if abs(SCF_E - E_old) < E_conv:
            break
        E_old = SCF_E

        # Compute new orbital guess
        epsilon, C_p = np.linalg.eigh(A @ F @ A)
        C = A @ C_p
        C_occ = C[:, :ndocc]
        D = np.einsum('pi,iq->pq', C_occ, C_occ.T, optimize=True)

        # MAXITER exceeded?
        if scf_iter == MAXITER:
            raise Exception("Maximum number of SCF iterations exceeded.")
    return SCF_E

# Definieer de variatieparameters
theta1 = 0  # Eerste waterstofatoom blijft op de x-as
afstand_values = np.arange(0.7, 1.0, 0.1)
theta2_values = np.arange(90, 110, 1)

# Matrix om resultaten op te slaan
energies_matrix = np.zeros((len(afstand_values), len(theta2_values)))

print(rhf('O 0.0 0.0 0.0; O 1.0 0.0 0.0'))


(8, 8)
-149.46916292716176
